In [8]:
import re
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Times Higher Education

## Общий рейтинг

С этим рейтингом тоже создаём объект `session`.

In [9]:
session = requests.session()
session.headers.update({
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ru',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'sec-ch-ua': '".Not/A)Brand";v="99", "Google Chrome";v="103", "Chromium";v="103"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
})
years = range(2020, 2024 + 1)

Дальше собираем данные по годам.

In [3]:
url_tpl = 'https://www.timeshighereducation.com/world-university-rankings/{year}/world-ranking'
ranking = pd.DataFrame(columns=['year'])

for year in years:
    html_url = url_tpl.format(year=year)
    html_resp = session.get(html_url)

    if html_resp.status_code == 200:
        html = html_resp.text
        soup = BeautifulSoup(html)
        scripts = soup.find_all('script')
        search_words = 'init_drupal_core_settings'
        drupal_settings_str = None

        for script in scripts:
            script_text = script.text

            if script_text.find(search_words) > -1:
                drupal_settings_str = script_text

        if drupal_settings_str:
            found = re.search(f"world_university_rankings_{year}_0__[\w\d]+\.json", drupal_settings_str)

            if found:
                data_url = f'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/{found[0]}'
                data_resp = session.get(data_url)

                if data_resp.status_code == 200:
                    data = data_resp.json()
                    chunk = pd.json_normalize(data['data'])
                    chunk['year'] = year
                    ranking = pd.concat([ranking, chunk], ignore_index=True)
                else:
                    print(f"{data_url} return status code {data_resp.status_code}")
    else:
        print(f"{html_url} return status code {html_resp.status_code}")

    # Бережём источник данных
    time.sleep(0.5)

ranking.to_csv(f"data/the_{min(years)}-{max(years)}.csv", index=False)
ranking.to_excel(f"data/the_{min(years)}-{max(years)}.xlsx", index=False)
ranking.head()

/tmp/ipykernel_218795/1317070433.py:32: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  ranking = pd.concat([ranking, chunk], ignore_index=True)
/tmp/ipykernel_218795/1317070433.py:32: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  ranking = pd.concat([ranking, chunk], ignore_index=True)
/tmp/ipykernel_218795/1317070433.py:32: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  ranking = pd.concat([ranking, chunk], ignore_index=True)
/tmp/ipykernel_218795/1317070433.py:32: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly

,year,rank_order,rank,name,scores_overall,scores_overall_rank,scores_teaching,scores_teaching_rank,scores_research,scores_research_rank,...,stats_pc_intl_students,stats_female_male_ratio,aliases,subjects_offered,closed,unaccredited,disabled,apply_link,cta_button.link,cta_button.text
0,2020,10,1,University of Oxford,95.4,10,90.5,6,99.6,1,...,41%,46 : 54,University of Oxford,"Geography,Chemistry,Chemical Engineering,Biolo...",False,False,False,https://www.timeshighereducation.com/student/r...,https://www.timeshighereducation.com/student/r...,Admissions Support
1,2020,20,2,California Institute of Technology,94.5,20,92.1,2,97.2,4,...,30%,34 : 66,California Institute of Technology caltech,"Computer Science,Chemistry,Chemical Engineerin...",False,False,False,https://www.timeshighereducation.com/student/r...,https://www.timeshighereducation.com/student/r...,Admissions Support
2,2020,30,3,University of Cambridge,94.4,30,91.4,4,98.7,2,...,37%,47 : 53,University of Cambridge,Politics & International Studies (incl Develop...,False,False,False,https://www.timeshighereducation.com/student/r...,https://www.timeshighereducation.com/student/r...,Admissions Support
3,2020,40,4,Stanford University,94.3,40,92.8,1,96.4,5,...,23%,43 : 57,Stanford University,"Computer Science,Communication & Media Studies...",False,False,False,NaN,https://www.timeshighereducation.com/student/r...,Admissions Support
4,2020,50,5,Massachusetts Institute of Technology,93.6,50,90.5,5,92.4,10,...,34%,39 : 61,Massachusetts Institute of Technology,"Architecture,Economics & Econometrics,Archaeol...",False,False,False,https://www.timeshighereducation.com/student/r...,https://www.timeshighereducation.com/student/r...,Admissions Support


Из полученных данных соберём количество 🇷🇺 российских университетов в рейтинге по годам.

In [5]:
ranking[ranking['location'] == 'Russian Federation'][['year', 'location']].\
    groupby(by='year').count().rename(columns={'location': 'count'})

,count
year,
2020,39
2021,48
2022,100
2023,103
2024,108


## Предметные

Код практически идентичен предыдущему, только появляется обход по каждому предмету (области).

In [10]:
url_subject_tpl = 'https://www.timeshighereducation.com/world-university-rankings/{year}/subject-ranking/{subject}'
ranking_subject = pd.DataFrame(columns=['year', 'subject'])
subject_map = [
    {'title': 'Arts & humanities', 'url_suffix': 'arts-and-humanities', 'file_prefix': 'arts_humanities_rankings'},
    {'title': 'Business & economics', 'url_suffix': 'business-and-economics', 'file_prefix': 'business_economics_rankings'},
    {'title': 'Clinical & health', 'url_suffix': 'clinical-pre-clinical-health', 'file_prefix': 'clinical_pre_clinical_health_ran'},
    {'title': 'Computer science', 'url_suffix': 'computer-science', 'file_prefix': 'computer_science_rankings'},
    {'title': 'Education', 'url_suffix': 'education', 'file_prefix': 'education_rankings'},
    {'title': 'Engineering', 'url_suffix': 'engineering-and-it', 'file_prefix': 'engineering_technology_rankings'},
    {'title': 'Law', 'url_suffix': 'law', 'file_prefix': 'law_rankings'},
    {'title': 'Life sciences', 'url_suffix': 'life-sciences', 'file_prefix': 'life_sciences_rankings'},
    {'title': 'Physical sciences', 'url_suffix': 'physical-sciences', 'file_prefix': 'physical_sciences_rankings'},
    {'title': 'Psychology', 'url_suffix': 'psychology', 'file_prefix': 'psychology_rankings'},
    {'title': 'Social sciences', 'url_suffix': 'social-sciences', 'file_prefix': 'social_sciences_rankings'}
]

for year in years:
    for subject in subject_map:
        html_url = url_subject_tpl.format(year=year, subject=subject['url_suffix'])
        html_resp = session.get(html_url)

        if html_resp.status_code == 200:
            html = html_resp.text
            soup = BeautifulSoup(html)
            scripts = soup.find_all('script')
            search_words = 'init_drupal_core_settings'
            drupal_settings_str = None

            for script in scripts:
                script_text = script.text

                if script_text.find(search_words) > -1:
                    drupal_settings_str = script_text

            if drupal_settings_str:
                found = re.search(f"{subject['file_prefix']}_{year}_0__[\w\d]+\.json", drupal_settings_str)

                if found:
                    data_url = f'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/{found[0]}'
                    data_resp = session.get(data_url)

                    if data_resp.status_code == 200:
                        data = data_resp.json()
                        chunk = pd.json_normalize(data['data'])
                        chunk[['year', 'subject']] = year, subject['title']
                        ranking_subject = pd.concat([ranking_subject, chunk], ignore_index=True)
                    else:
                        print(f"{data_url} return status code {data_resp.status_code}")
        else:
            print(f"{html_url} return status code {html_resp.status_code}")

        # Бережём источник данных
        time.sleep(0.5)

ranking_subject.to_csv(f"data/the_{min(years)}-{max(years)}_subject.csv", index=False)
ranking_subject.to_excel(f"data/the_{min(years)}-{max(years)}_subject.xlsx", index=False)
ranking_subject.head()

/tmp/ipykernel_218795/82309728.py:46: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  ranking_subject = pd.concat([ranking_subject, chunk], ignore_index=True)
/tmp/ipykernel_218795/82309728.py:46: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  ranking_subject = pd.concat([ranking_subject, chunk], ignore_index=True)
/tmp/ipykernel_218795/82309728.py:46: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  ranking_subject = pd.concat([ranking_subject, chunk], ignore_index=True)
/tmp/ipykernel_218795/82309728.py:46: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in re

,year,subject,rank_order,rank,name,scores_overall,scores_overall_rank,scores_citations,scores_citations_rank,scores_industry_income,...,stats_pc_intl_students,stats_female_male_ratio,aliases,subjects_offered,closed,unaccredited,disabled,cta_button.link,cta_button.text,apply_link
0,2020,Arts & humanities,10,1,Stanford University,89.9,10,80.9,6,72.4,...,23%,43 : 57,Stanford University,"History, Philosophy & Theology,Art, Performing...",False,False,False,https://www.timeshighereducation.com/student/r...,Admissions Support,NaN
1,2020,Arts & humanities,20,2,University of Cambridge,86.7,20,62.8,144,58.8,...,37%,47 : 53,University of Cambridge,"History, Philosophy & Theology,Architecture,Ar...",False,False,False,https://www.timeshighereducation.com/student/r...,Admissions Support,https://www.timeshighereducation.com/student/r...
2,2020,Arts & humanities,30,3,University of Oxford,86.4,30,68.1,85,35.2,...,41%,46 : 54,University of Oxford,"Art, Performing Arts & Design,History, Philoso...",False,False,False,https://www.timeshighereducation.com/student/r...,Admissions Support,https://www.timeshighereducation.com/student/r...
3,2020,Arts & humanities,40,4,Massachusetts Institute of Technology,85.8,40,76.3,18,56.5,...,34%,39 : 61,Massachusetts Institute of Technology,"Architecture,Archaeology,Languages, Literature...",False,False,False,https://www.timeshighereducation.com/student/r...,Admissions Support,https://www.timeshighereducation.com/student/r...
4,2020,Arts & humanities,50,5,Harvard University,84.3,50,68.3,84,36.0,...,24%,49 : 51,Harvard University,"Architecture,History, Philosophy & Theology,La...",False,False,False,https://www.timeshighereducation.com/student/r...,Admissions Support,https://www.timeshighereducation.com/student/r...


Для пример давайте посмотрим динамику университетов некоторых стран в предметных рейтингах.

In [21]:
pd.pivot_table(
    ranking_subject[ranking_subject['location'].isin([
        'Russian Federation', 'China',
        'India', 'Germany', 'Turkey',
        'United Kingdom', 'United States',
        'France', 'Brazil', 'Iran', 
        'Indonesia', 'Pakistan', 'Japan',
        'Thailand', 'Italy'
    ])],
    index='location',
    columns='year',
    values='name',
    aggfunc='count'
).sort_values(by=[2024], ascending=False)

year,2020,2021,2022,2023,2024
location,,,,,
United States,1385,1470,1499,1495,1447
United Kingdom,695,737,757,785,793
China,396,468,513,528,499
Japan,322,331,351,347,360
Italy,271,288,319,336,352
Brazil,243,287,307,341,332
Germany,284,286,310,320,320
Turkey,122,145,181,216,271
India,153,172,195,221,270
